In [12]:
pip install scikit-learn

  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
    --------------------------------------- 0.3/11.0 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.0 MB 3.3 MB/s eta 0:00:03
   ------ --------------------------------- 1.8/11.0 MB 4.0 MB/s eta 0:00:03
   ---------- ----------------------------- 2.9/11.0 MB 4.1 MB/s eta 0:00:02
   -------------- ------------------------- 3.9/11.0 MB 4.3 MB/s eta 0:00:02
   --------------- ------------------------ 4.2/11.0 MB 4.3 MB/s eta 0:00:02
   ------------------- -------------------- 5.2/11.0 MB 3.8 MB/s eta 0:00:02
   -------------------- ------------------- 5.5/11.0 MB 3.8 MB/s eta 0:00:02
   ---------------------- ----------------- 6.3/11.0 MB 3.4 MB/s eta 0:00:02
   ------------------------- -------------- 7.1/11.0 MB 3.5 MB/s eta 0:00:02
   -------------------------

### 컬럼 확인

In [21]:
import pandas as pd

# CSV 파일 불러오기 (파일 경로를 올바르게 설정해야 합니다)
file_path = "직무정보_cleaned.csv"
df = pd.read_csv(file_path)

# 컬럼명 확인
print(df.columns.tolist())

['직무명', '직무개요', '수행직무', '유사명칭', '관련직업', '자격/면허', '고용직업분류', '표준직업분류', '표준산업분류', '권장 교육수준', '권장 숙련기간_최소', '권장 숙련기간_최대', '직무관련성_자료', '직무관련성_사람', '직무관련성_사물', '작업강도_아주 힘든 작업', '작업강도_보통 작업', '작업강도_아주 가벼운 작업', '작업강도_힘든 작업', '작업강도_가벼운 작업', '육체활동_균형감각', '육체활동_언어력', '육체활동_청각', '육체활동_시각', '육체활동_손사용', '육체활동_웅크림', '작업환경_소음·진동', '작업환경_다습', '작업환경_저온', '작업환경_고온', '작업환경_위험내재', '작업환경_대기환경미흡', '작업장소_실내·외', '작업장소_실외', '작업장소_실내']


### 컬럼별 전처리

In [25]:
import pandas as pd
import re
from konlpy.tag import Okt
from sklearn.preprocessing import MultiLabelBinarizer


# 열 이름: 직무개요, 수행직무
# 상태: 요약 필요
# 처리: 함수에 주석으로 설명
okt = Okt()

# 불용어 목록 로드: 해당 열에서 부사와 형용사만 추출된 데이터
def load_stop_words_from_file(file_path_2):

    file_path_2 = "../데이터/직업정보_부사 및 형용사.csv"
    df = pd.read_csv(file_path_2)
    return df['단어'].tolist()

# 전처리 함수
def preprocess_job_descriptions_refined(text, stop_words):
    """
    주어진 텍스트에서 지정된 불용어를 제거하고 전처리합니다.
    """
    # 입력이 None이거나 문자열이 아닌 경우 빈 문자열 반환
    if not isinstance(text, str):
        return ""
    
    # 1. '을/를 한다' 제거 (정규 표현식 사용)
    text = re.sub(r'(\w+)을 한다\b', r'\1', text) 
    text = re.sub(r'(\w+)를 한다\b', r'\1', text) 
    text = re.sub(r'\b한다\b', '', text)  # 띄어쓰기 된 "한다"만 제거

    # 2. 붙어있는 "한다" 제거 (예: '조작한다' -> '조작')
    text = re.sub(r'(\w+)한다\b', r'\1', text)  # "단어 + 한다" 형태에서 '한다'만 제거

    # 3. 불용어 목록에 있는 단어 제거
    words = text.split()
    refined_words = [word for word in words if word not in stop_words]

    # 4. 괄호로 이루어진 내용 제거 (괄호와 그 내부의 모든 내용을 제거)
    refined_text = ' '.join(refined_words)
    refined_text = re.sub(r'\(.*?\)', '', refined_text)
    
    # 5. 특수 기호 제거 및 중복된 공백 제거
    refined_text = re.sub(r'[^가-힣\s]', '', refined_text)  # 한글과 공백을 제외한 모든 기호 제거
    refined_text = re.sub(r'\s+', ' ', refined_text).strip()
    
    return refined_text

# 데이터 불러오기
stop_words_file = "../데이터/직업정보_부사 및 형용사.csv"

# 불용어 로드
stop_words = load_stop_words_from_file(stop_words_file)

# 데이터 로드
df = pd.read_csv("../데이터/직업정보크롤링_all.csv")

# 개선된 전처리 적용
df['직무개요'] = df['직무개요'].apply(lambda text: preprocess_job_descriptions_refined(text, stop_words))
df['수행직무'] = df['수행직무'].apply(lambda text: preprocess_job_descriptions_refined(text, stop_words))

# 이후의 기존 처리 과정 계속 진행

# 열 이름: 정규교육
# 상태: 요약 필요
# 처리: 1) 교육의 정확한 기간은 제외, 중졸/고졸/대졸/전문대졸 로 나눔
# 처리: 2) 해당 열의 이름이 불명확하므로, '권장 교육수준'으로 변경
def extract_education(text):
    if pd.isna(text):
        return text
    match = re.search(r'\((.*?)\)', text)
    if match:
        return match.group(1).replace('정도', '').strip()
    return None

df['권장 교육수준'] = df['정규교육'].apply(extract_education)
df.drop(columns=['정규교육'], inplace=True)

# 열 이름: 숙련기간
# 상태: 분류 필요
# 처리: 1) "권장 숙련기간_최소", "권장 숙련기간_최대" 로 스플릿
# 처리: 2) n개월/년 초과 ~ n개월/년 이하로 이루어져 있으므로, 전자는 최소 행에, 후자는 최대 행에 넣음
def split_experience(text):
    if pd.isna(text):
        return None, None
    match = re.search(r'(\d+개월|\d+년) 초과 ~ (\d+개월|\d+년) 이하', text)
    if match:
        return match.group(1), match.group(2)
    return None, None

df['권장 숙련기간_최소'], df['권장 숙련기간_최대'] = zip(*df['숙련기간'].apply(split_experience))

# 열 이름: 직무기능
# 상태: 분류 필요
# 처리: 1) 자료/사람/사물에 따라 3가지 열로 나눔
# 처리: 2) 괄호에 해당하는 세부사항을 행으로 넣음
# 처리: 3) 열 이름이 불분명하므로 '직무 관련성_{}'으로 변경
def extract_job_function(text):
    if pd.isna(text):
        return None, None, None
    data_match = re.search(r'자료\s*\((.*?)\)', text)
    people_match = re.search(r'사람\s*\((.*?)\)', text)
    object_match = re.search(r'사물\s*\((.*?)\)', text)
    return (data_match.group(1) if data_match else None,
            people_match.group(1) if people_match else None,
            object_match.group(1) if object_match else None)

df['직무관련성_자료'], df['직무관련성_사람'], df['직무관련성_사물'] = zip(*df['직무기능'].apply(extract_job_function))

# 열 이름: 비고
# 상태: 삭제 필요한 행 존재
# 처리: '-' 또는 결측치 외의 텍스트는 전부 해당 직무가 사라져가거나 없어졌다는 의미를 담고있음
# 따라서 텍스트가 있는 행은 삭제
# '비고' 열에서 '-' 또는 결측치가 아닌 행을 필터링하여 삭제하고 해당 열도 삭제
df = df[df['비고'].isna() | (df['비고'] == '-')]

def create_unique_dummy_variables(column_name):
    """
    특정 열의 값에서 중복된 항목을 제거하고 고유 값만을 기준으로 더미 변수를 생성.
    """
    # 1. 결측치 및 '-' 값 처리
    df[column_name] = df[column_name].fillna('')
    df[column_name] = df[column_name].replace({'-': ''})
    
    # 2. 각 텍스트에서 \r 제거
    df[column_name] = df[column_name].apply(lambda x: x.replace('\r', '').split('\n'))
    
    # 3. 고유한 작업 환경 목록 만들기
    unique_values = set([item for sublist in df[column_name] for item in sublist if item])
    
    # 4. 더미 변수를 고유한 값으로 생성
    for value in unique_values:
        df[f'{column_name}_{value}'] = df[column_name].apply(lambda x: int(value in x))
    
    # 중간 결과 열 삭제
    df.drop(columns=[column_name], inplace=True)

# 처리 대상 열에 대해 고유한 값으로 더미 변수화 적용
create_unique_dummy_variables('작업강도')
create_unique_dummy_variables('육체활동')
create_unique_dummy_variables('작업환경')
create_unique_dummy_variables('작업장소')

# 필요 없는 열 삭제
df = df.drop(columns=['숙련기간', '직무기능', '비고', '조사연도'])

# '-' 또는 빈 문자열: NaN로 처리
df.replace({'-': pd.NA, '': pd.NA}, inplace=True)

# 저장
df.to_csv("직무정보_cleaned_4.csv", index=False, encoding='utf-8-sig')


### 정제 후 컬럼 종류

'직무명'

'직무개요'

'수행직무'

'유사명칭'

'관련직업'

'자격/면허'

'고용직업분류', '표준직업분류', '표준산업분류'

'권장 교육수준'

'권장 숙련기간_최소', '권장 숙련기간_최대'

'직무관련성_자료', '직무관련성_사람', '직무관련성_사물'

<br>
[태그로 선택할 수 있도록 분류: 직무관련성, 직업강도, 육체활동, 작업환경]


'작업강도_아주 힘든 작업', '작업강도_보통 작업', '작업강도_아주 가벼운 작업', '작업강도_힘든 작업', '작업강도_가벼운 작업' 

'육체활동_균형감각', '육체활동_언어력', '육체활동_청각', '육체활동_시각', '육체활동_손사용', '육체활동_웅크림',

'작업환경_소음·진동', '작업환경_다습', '작업환경_저온', '작업환경_고온', '작업환경_위험내재', '작업환경_대기환경미흡' 

'작업장소_실내·외', '작업장소_실외', '작업장소_실내'

In [26]:
# 각 카테고리별 더미 변수 그룹 확인
categories = {
    '작업강도': [col for col in df.columns if col.startswith('작업강도_')],
    '육체활동': [col for col in df.columns if col.startswith('육체활동_')],
    '작업환경': [col for col in df.columns if col.startswith('작업환경_')],
    '작업장소': [col for col in df.columns if col.startswith('작업장소_')]
}

# 각 카테고리별로 모든 값이 0인 행이 있는지 확인
for category, columns in categories.items():
    zero_rows = df[columns].sum(axis=1) == 0  # 각 열의 합이 0인 행 확인
    count_zero_rows = zero_rows.sum()  # 모든 값이 0인 행의 수
    
    print(f"{category} 카테고리에서 모든 값이 0인 행의 수: {count_zero_rows}")
    if count_zero_rows > 0:
        print(f"모든 값이 0인 행의 인덱스: {df[zero_rows].index.tolist()}")


작업강도 카테고리에서 모든 값이 0인 행의 수: 1
모든 값이 0인 행의 인덱스: [836]
육체활동 카테고리에서 모든 값이 0인 행의 수: 3847
모든 값이 0인 행의 인덱스: [1, 4, 6, 9, 10, 12, 13, 15, 17, 19, 21, 22, 23, 24, 26, 28, 32, 33, 34, 35, 36, 37, 47, 49, 51, 53, 54, 55, 56, 57, 62, 63, 64, 65, 66, 69, 74, 77, 78, 79, 80, 81, 82, 85, 86, 87, 89, 90, 91, 92, 93, 95, 96, 100, 101, 102, 103, 105, 107, 115, 116, 118, 119, 120, 122, 124, 126, 129, 130, 133, 138, 139, 140, 142, 144, 145, 146, 147, 150, 151, 152, 154, 158, 160, 161, 162, 163, 164, 165, 166, 168, 169, 170, 171, 172, 173, 174, 175, 176, 178, 179, 180, 182, 183, 184, 186, 187, 188, 189, 190, 192, 194, 195, 196, 197, 198, 199, 202, 203, 207, 208, 209, 210, 211, 212, 213, 214, 215, 226, 227, 228, 231, 232, 233, 234, 235, 236, 237, 238, 239, 241, 243, 244, 246, 249, 250, 260, 262, 263, 264, 265, 266, 267, 269, 272, 273, 274, 275, 276, 277, 278, 279, 280, 282, 283, 284, 285, 286, 287, 288, 290, 292, 294, 295, 296, 297, 299, 301, 303, 304, 305, 306, 307, 312, 313, 314, 315, 316, 319, 320, 321, 

육체활동, 작업환경은 특이사항이 없는 경우 표기되지 않는 직무가 다수임

따라서 이 경우는 "{}_특이사항 없음" 행을 만들어 1로 표기

직업강도와 직업장소가 표기되지 않는 행은 직접 확인 후 해결

In [27]:
# 각 카테고리별 더미 변수 그룹 확인
categories = {
    '작업강도': [col for col in df.columns if col.startswith('작업강도_')],
    '육체활동': [col for col in df.columns if col.startswith('육체활동_')],
    '작업환경': [col for col in df.columns if col.startswith('작업환경_')],
    '작업장소': [col for col in df.columns if col.startswith('작업장소_')]
}

# 육체활동과 작업환경 카테고리에서 모든 값이 0인 행 확인 및 처리
for category in ['육체활동', '작업환경']:
    columns = categories[category]
    
    # 모든 값이 0인 행 확인
    zero_rows = (df[columns].sum(axis=1) == 0)
    
    # 새로운 열 "육체활동_특이사항 없음"을 생성하고 모든 값이 0인 행에 대해 1로 처리
    if category == '육체활동':
        df['육체활동_특이사항 없음'] = 0  # 기본값 0으로 설정
        df.loc[zero_rows, '육체활동_특이사항 없음'] = 1  # 모든 값이 0인 행에 대해 1로 설정

    elif category == '작업환경':
        df['작업환경_특이사항 없음'] = 0  # 기본값 0으로 설정
        df.loc[zero_rows, '작업환경_특이사항 없음'] = 1  # 모든 값이 0인 행에 대해 1로 설정

# 결과 확인
print("육체활동과 작업환경 카테고리에서 특이사항 없음 열 추가 완료.")


육체활동과 작업환경 카테고리에서 특이사항 없음 열 추가 완료.


In [28]:
# 각 카테고리별 더미 변수 그룹 확인
categories = {
    '작업강도': [col for col in df.columns if col.startswith('작업강도_')],
    '육체활동': [col for col in df.columns if col.startswith('육체활동_')],
    '작업환경': [col for col in df.columns if col.startswith('작업환경_')],
    '작업장소': [col for col in df.columns if col.startswith('작업장소_')]
}

# 각 카테고리별로 모든 값이 0인 행이 있는지 확인
for category, columns in categories.items():
    zero_rows = df[columns].sum(axis=1) == 0  # 각 열의 합이 0인 행 확인
    count_zero_rows = zero_rows.sum()  # 모든 값이 0인 행의 수
    
    print(f"{category} 카테고리에서 모든 값이 0인 행의 수: {count_zero_rows}")
    if count_zero_rows > 0:
        print(f"모든 값이 0인 행의 인덱스: {df[zero_rows].index.tolist()}")


작업강도 카테고리에서 모든 값이 0인 행의 수: 1
모든 값이 0인 행의 인덱스: [836]
육체활동 카테고리에서 모든 값이 0인 행의 수: 0
작업환경 카테고리에서 모든 값이 0인 행의 수: 0
작업장소 카테고리에서 모든 값이 0인 행의 수: 2
모든 값이 0인 행의 인덱스: [2560, 4332]


In [29]:
df.to_csv("직무정보_cleaned_5.csv", index=False, encoding='utf-8-sig')

- 작업강도 카테고리에서 모든 값이 0인 행

    : 유사한 직무의 직업강도로 동일하게 채움([836]아주 가벼움)

- 작업장소 카테고리에서 모든 값이 0인 행

    : 유사한 직업의 작업장소로 동일하게 채움



In [34]:
import pandas as pd

# 데이터 불러오기

df = pd.read_csv("직무정보_cleaned_5.csv")

# 각 카테고리별 더미 변수 그룹 확인
categories = {
    '작업강도': [col for col in df.columns if col.startswith('작업강도_')],
    '육체활동': [col for col in df.columns if col.startswith('육체활동_')],
    '작업환경': [col for col in df.columns if col.startswith('작업환경_')],
    '작업장소': [col for col in df.columns if col.startswith('작업장소_')]
}

# 각 카테고리별로 모든 값이 0인 행이 있는지 확인
for category, columns in categories.items():
    zero_rows = df[columns].sum(axis=1) == 0  # 각 열의 합이 0인 행 확인
    count_zero_rows = zero_rows.sum()  # 모든 값이 0인 행의 수
    
    print(f"{category} 카테고리에서 모든 값이 0인 행의 수: {count_zero_rows}")
    if count_zero_rows > 0:
        print(f"모든 값이 0인 행의 인덱스: {df[zero_rows].index.tolist()}")


작업강도 카테고리에서 모든 값이 0인 행의 수: 0
육체활동 카테고리에서 모든 값이 0인 행의 수: 0
작업환경 카테고리에서 모든 값이 0인 행의 수: 0
작업장소 카테고리에서 모든 값이 0인 행의 수: 0


### unique 확인

In [3]:
import pandas as pd

# CSV 파일 불러오기
file_path = '../데이터/직업정보크롤링_cleaned.csv'  # 파일 경로를 여기에 입력
df = pd.read_csv(file_path)

# 확인하고 싶은 열 이름 리스트
columns_to_check = ['직무관련성_자료', '직무관련성_사람', '직무관련성_사물', '권장 교육수준']  # 확인하고 싶은 열 이름을 여기에 입력

# 지정한 열별 고유값의 종류 출력
for column in columns_to_check:
    if column in df.columns:  # 열이 실제 데이터프레임에 있는지 확인
        unique_values = df[column].unique()
        print(f"열 이름: {column}")
        print(f"고유값 종류: {len(unique_values)}")
        print(f"고유값 목록: {unique_values}\n")
    else:
        print(f"열 이름 '{column}'이 데이터프레임에 존재하지 않습니다.\n")


열 이름: 직무관련성_자료
고유값 종류: 8
고유값 목록: ['비교' '수집' '종합' '분석' '조정' '관련없음' '계산' '기록']

열 이름: 직무관련성_사람
고유값 종류: 9
고유값 목록: ['말하기-신호' '관련없음' '협의' '설득' '감독' '자문' '서비스 제공' '교육' '오락제공']

열 이름: 직무관련성_사물
고유값 종류: 9
고유값 목록: ['수동조작' '제어조작' '정밀작업' '조작운전' '단순작업' '유지' '관련없음' '투입-인출' '설치']

열 이름: 권장 교육수준
고유값 종류: 6
고유값 목록: ['중졸' '고졸' '대졸' '전문대졸' '대학원 이상' '초졸 이하']

